# Final Project Title


# Column Descriptions! 

YEAR Year of the Flight Trip 

MONTH Month of the Flight Trip

DAY Day of the Flight Trip 

DAY_OF_WEEK Day of week of the Flight Trip

AIRLINE Airline Identifier

FLIGHT_NUMBER Flight Identifier

TAIL_NUMBER Aircraft Identifier

ORIGIN_AIRPORT Starting Airport

DESTINATION_AIRPORT  Destination Airport

SCHEDULED_DEPARTURE  Planned Departure Time

DEPARTURE_TIMEWHEEL_OFF - TAXI_OUT

DEPARTURE_DELAY  Total Delay on Departure

TAXI_OUT The time duration elapsed between departure from the origin airport gate and wheels off

WHEELS_OFFThe time point that the aircraft's wheels leave the ground

SCHEDULED_TIMEPlanned time amount needed for the flight trip

ELAPSED_TIMEAIR_TIME+TAXI_IN+TAXI_OUT

AIR_TIMEThe time duration between wheels_off and wheels_on time

DISTANCEDistance between two airports

WHEELS_ON The time point that the aircraft's wheels touch on the ground

TAXI_IN The time duration elapsed between wheels-on and gate arrival at the destination airport

SCHEDULED_ARRIVAL Planned arrival time

ARRIVAL_TIMEWHEELS_ON+TAXI_IN

ARRIVAL_DELAY 

ARRIVAL_TIME-SCHEDULED_ARRIVAL

DIVERTED Aircraft landed on airport that out of schedule

CANCELLED Flight Cancelled (1 = cancelled)

CANCELLATION_REASON Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security

AIR_SYSTEM_DELAY Delay caused by air system

SECURITY_DELAY Delay caused by security

AIRLINE_DELAY Delay caused by the airline

LATE_AIRCRAFT_DELAY Delay caused by aircraft

WEATHER_DELAY Delay caused by weather

In [24]:
#read in file as dataframe 
# import pyspark modules
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
from pyspark.mllib.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
import pandas as pd
import os
import pyspark.sql.types as typ
import pyspark.sql.functions as F

In [25]:
from pyspark.sql import SparkSession 
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("app") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '2') \
    .config('spark.cores.max', '2') \
    .config("spark.driver.memory",'4g') \
    .getOrCreate()

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

### Start of APT edit

In [26]:
path_to_data = os.path.join("/home/jovyan/FlightDelay/flights.csv")

read into rdd

In [27]:
delay_rdd = sc.textFile(path_to_data)

In [5]:
#delay_rdd.take(3)

read into spark dataframe

In [7]:
delay_df = spark.read.format("csv").option("header", "true").option("inferschema","true").load(path_to_data)

In [28]:
delay_df.show(3)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+
|2015|    1|  1|          4|     AS|           98|     N407AS|           ANC|                SEA|            

In [29]:
delay_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [30]:
rowcount=delay_df.count()
rowcount

5819079

In [31]:
delay_df.describe(['DEPARTURE_DELAY', 'ARRIVAL_DELAY']).show()

+-------+-----------------+-----------------+
|summary|  DEPARTURE_DELAY|    ARRIVAL_DELAY|
+-------+-----------------+-----------------+
|  count|          5732926|          5714008|
|   mean|9.370158275198389|4.407057357987598|
| stddev|37.08094249678714|39.27129709388608|
|    min|              -82|              -87|
|    max|             1988|             1971|
+-------+-----------------+-----------------+



# Check for Duplicates

In [32]:
print('rows = {}'.format(rowcount))

rows = 5819079


In [33]:
print('rows = {}'.format(delay_df.distinct().count()))

rows = 5819079


Appears there are no duplicated entries

# Check for Missing Values 

In [35]:
# calculates percent of missing values in ecah column! 
missing = delay_df.agg(*[
    (1-F.count(c) / F.count('*')).alias(c + '_missing')
    for c in delay_df.columns
]).show()

+------------+-------------+-----------+-------------------+---------------+---------------------+--------------------+----------------------+---------------------------+---------------------------+----------------------+-----------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------------+-------------------+-------------------+-------------------------+--------------------+---------------------+----------------+-----------------+
|YEAR_missing|MONTH_missing|DAY_missing|DAY_OF_WEEK_missing|AIRLINE_missing|FLIGHT_NUMBER_missing| TAIL_NUMBER_missing|ORIGIN_AIRPORT_missing|DESTINATION_AIRPORT_missing|SCHEDULED_DEPARTURE_missing|DEPARTURE_TIME_missing|DEPARTURE_DELAY_missing|    TAXI_OUT_missing|  WHEELS_OFF_missing|SCHEDULED_TIME_missing|ELAPSED_TIME_missing|    AIR_TIME_missing|DISTANCE_missing|  WHEELS_ON_missing|    TAXI_IN_missing|SCHEDULED_ARRIVAL_missing|ARRIVAL_TIME_missing|ARRIVAL_DELAY_missing|DIVERTED

Last 6 columns appear to have very large percentage of missing values: 

# Lauren - where do you see this?

CANCELLATION_REASON_missing, AIR_SYSTEM_DELAY_missing, SECURITY_DELAY_missing, 
AIRLINE_DELAY_missing, LATE_AIRCRAFT_DELAY_missing, WEATHER_DELAY_missing 

Should I drop all of these columns? 

# Drop columns 

In [40]:
cols_to_drop = ['WEATHER_DELAY', 'SECURITY_DELAY', 'AIR_SYSTEM_DELAY','AIRLINE_DELAY', 
                'LATE_AIRCRAFT_DELAY', 'CANCELLATION_REASON'] 

delay_df = delay_df.drop(*cols_to_drop)

In [18]:
# Drop records where at least 3 columns have NULL values 

delay_df = delay_df.dropna(thresh=3)

Doesn't change the count of the DF, so no rows with atleast 3 missing columns 

In [41]:
delay_df.count()

5819079

# Impute Missing values 

In [42]:
drop_cols = cols_to_exclude = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK',
                   'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DIVERTED', 
                   'CANCELLED', 'DESTINATION_AIRPORT', 'AIRLINE', 
                   'FLIGHT_NUMBER', 'TAIL_NUMBER']


df_impute = delay_df.drop(*drop_cols)
means = df_impute.agg(*[F.mean(c).alias(c) \
                                for c in df_impute.columns]) \
                                .toPandas().to_dict('records')[0]

# Outliers 

In [45]:
cols_to_exclude = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK',
                   'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DIVERTED', 
                   'CANCELLED', 'DESTINATION_AIRPORT', 'AIRLINE', 
                   'FLIGHT_NUMBER', 'TAIL_NUMBER'] 

df_for_outlier_calc = delay_df.drop(*cols_to_exclude)

cols = [c for c in df_for_outlier_calc.columns]
bounds = {}

In [46]:
for col in cols:
    quantiles = df_for_outlier_calc.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    
    bounds[col] = [
        quantiles[0] - 1.5 * IQR,
        quantiles[1] + 1.5 * IQR
    ]

KeyboardInterrupt: 

In [ ]:
# append outlier indicator to data table \

outliers = df_for_outlier_calc.select(*['TAIL_NUMBER'] + [
 (
 (df_for_outlier_calc[c] < bounds[c][0]) | (df_for_outlier_calc[c] > bounds[c][1]))
    .alias(c + '_outlier') for c in cols
])

In [ ]:
outliers.show(4)

# Bucketizer (practice)

In [47]:
from pyspark.ml.feature import Bucketizer
from pyspark.sql import Row

splits = [-9999999, -0.5, 0.0, 0.5, 9999999]

data = [-999.9, -0.5, -0.3, 0.0, 0.2, 999.9]

data= list(map(lambda x: Row(numbers=x), data))

data

[Row(numbers=-999.9),
 Row(numbers=-0.5),
 Row(numbers=-0.3),
 Row(numbers=0.0),
 Row(numbers=0.2),
 Row(numbers=999.9)]

In [48]:
dataFrame = sqlCtx.createDataFrame(data, ["numbers"])

In [49]:
bucketizer = Bucketizer() \
  .setInputCol("numbers") \
  .setOutputCol("bucketedFeatures") \
  .setSplits(splits)


#Transform original data into its bucket index.
bucketedData = bucketizer.transform(dataFrame)

print("Bucketizer output with ${bucketizer.getSplits.length-1} buckets")
bucketedData.show()

Bucketizer output with ${bucketizer.getSplits.length-1} buckets
+-------+----------------+
|numbers|bucketedFeatures|
+-------+----------------+
| -999.9|             0.0|
|   -0.5|             1.0|
|   -0.3|             1.0|
|    0.0|             2.0|
|    0.2|             2.0|
|  999.9|             3.0|
+-------+----------------+



# Bucketizer (application)

In [54]:
delay_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [56]:
delay_df.select(["DEPARTURE_TIME", "SCHEDULED_DEPARTURE", "ARRIVAL_TIME", "SCHEDULED_TIME"]).show(10)

+--------------+-------------------+------------+--------------+
|DEPARTURE_TIME|SCHEDULED_DEPARTURE|ARRIVAL_TIME|SCHEDULED_TIME|
+--------------+-------------------+------------+--------------+
|          2354|                  5|         408|           205|
|             2|                 10|         741|           280|
|            18|                 20|         811|           286|
|            15|                 20|         756|           285|
|            24|                 25|         259|           235|
|            20|                 25|         610|           217|
|            19|                 25|         509|           181|
|            44|                 30|         753|           273|
|            19|                 30|         532|           195|
|            33|                 30|         656|           221|
+--------------+-------------------+------------+--------------+
only showing top 10 rows



In [58]:
delay_splits = [0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400]

In [66]:
# Does the job, quickly too, but not very elegantly. Look into how to bucketize groups of columns

deptime_bucketizer = Bucketizer() \
  .setInputCol("DEPARTURE_TIME") \
  .setOutputCol("B_DEPARTURE_TIME") \
  .setSplits(delay_splits)

scheddep_bucketizer = Bucketizer() \
  .setInputCol("SCHEDULED_DEPARTURE") \
  .setOutputCol("B_SCHEDULED_DEPARTURE") \
  .setSplits(delay_splits)

arrtime_bucketizer = Bucketizer() \
  .setInputCol("ARRIVAL_TIME") \
  .setOutputCol("B_ARRIVAL_TIME") \
  .setSplits(delay_splits)

schedarr_bucketizer = Bucketizer() \
  .setInputCol("SCHEDULED_ARRIVAL") \
  .setOutputCol("B_SCHEDULED_ARRIVAL") \
  .setSplits(delay_splits)


In [67]:
#Transform original data into its bucket index.
delay_df_b = deptime_bucketizer\
               .transform(scheddep_bucketizer\
               .transform(arrtime_bucketizer\
               .transform(schedarr_bucketizer\
               .transform(delay_df))))

In [68]:
delay_df_b.select(["B_DEPARTURE_TIME", "B_SCHEDULED_DEPARTURE", "B_ARRIVAL_TIME", "B_SCHEDULED_ARRIVAL"]).show(10)

+----------------+---------------------+--------------+-------------------+
|B_DEPARTURE_TIME|B_SCHEDULED_DEPARTURE|B_ARRIVAL_TIME|B_SCHEDULED_ARRIVAL|
+----------------+---------------------+--------------+-------------------+
|             7.0|                  0.0|           1.0|                1.0|
|             0.0|                  0.0|           2.0|                2.0|
|             0.0|                  0.0|           2.0|                2.0|
|             0.0|                  0.0|           2.0|                2.0|
|             0.0|                  0.0|           0.0|                1.0|
|             0.0|                  0.0|           2.0|                2.0|
|             0.0|                  0.0|           1.0|                1.0|
|             0.0|                  0.0|           2.0|                2.0|
|             0.0|                  0.0|           1.0|                1.0|
|             0.0|                  0.0|           2.0|                2.0|
+-----------